In [13]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import torchvision

import numpy as np
import random
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns


In [6]:
class NN(nn.Module):
    def __init__(self):
        super(NN, self).__init__()
        self.fc1 = nn.Linear(32*32*3, 50)
        self.fc2 = nn.Linear(50, 20)
        self.fc3 = nn.Linear(20, 7)

    def forward(self, x):
        x = nn.Flatten(x)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.fc3(x)
        x = F.softmax(x)
        return x

In [11]:
# read csv file
Train_csv = pd.read_csv('archive/Train.csv')

# 43개 중 ClassId가 0, 1, 2, 3, 4, 5, 7, 8 인 데이터만 추출
Train_csv = Train_csv[Train_csv['ClassId'].isin([0, 1, 2, 3, 4, 5, 7, 8])]
Train_csv

# 각 ClassId별로 데이터 개수 확인
Train_csv['ClassId'].value_counts()

2    2250
1    2220
4    1980
5    1860
7    1440
3    1410
8    1410
0     210
Name: ClassId, dtype: int64

In [ ]:
dataloader = torch.utils.data.DataLoader(Train_csv, batch_size=4, shuffle=True, num_workers=2)

# get some random training images
dataiter = iter(dataloader)
images, labels = dataiter.next()

# show images
plt.imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % labels[j] for j in range(4)))

# model
model = NN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)

# train
for epoch in range(2):
    running_loss = 0.0
    for i, data in enumerate(dataloader, 0):
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        if i % 2000 == 1999:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

